In [ ]:
import datetime as dt
import time
import random
import logging
from optibook.synchronous_client import Exchange
from optibook.common_types import InstrumentType, OptionKind
from dual_listing import dl_tools as dl
from options_quoter import black_scholes as bs
from options_quoter import options_quoter as oq
from options_quoter import libs
import numpy as np
import pandas as pd
import math
import multiprocessing as mp

In [ ]:
exchange = Exchange()
exchange.connect()
logging.getLogger('client').setLevel('ERROR')

In [ ]:
stock_id_ref = 'NVDA'
stock_id_dual = f'{stock_id_ref}_DUAL'
all_insts = list(exchange.get_instruments())
futures = set([ 'NVDA_202306_F', 'NVDA_202309_F','NVDA_202312_F'])
options = set(['NVDA_202306_050C', 'NVDA_202306_040P', 'NVDA_202306_040C', 'NVDA_202306_030P', 'NVDA_202306_030C', 'NVDA_202306_020C', 'NVDA_202306_050P', 'NVDA_202306_020P'])
stocks = set(['CSCO', 'ING', 'NVDA', 'NVDA_DUAL', 'PFE', 'SAN', 'SAN_DUAL'])
calls = set(['NVDA_202306_050C', 'NVDA_202306_040C', 'NVDA_202306_030C', 'NVDA_202306_020C'])
puts = set(['NVDA_202306_040P', 'NVDA_202306_030P','NVDA_202306_050P', 'NVDA_202306_020P'])
NVDA_derivs = futures.union(options).union(set(['NVDA_DUAL']))
SAN_derivs = set(['SAN_DUAL'])
sigma = 3
int_rate = 0.03

## Passive Strategy

In [ ]:
def obtain_passive_list(exchange,unchanged_list,insts_info,stock_id_ref,delta_side='positive'):
    '''
    takes in the dictionary of dictionaries insts_info, containing best price & ideal price & my outstanding volumes
    side is bid/ask (where we want to insert our limit)
    Returns a list of lists, where each sublist is [inst_id, limit_price to be put on]
    We want instruments with the highest spread to be placed first; if they have the same spread, we want those with the most available inventory.
    '''
    passive_list = []    
    ref_pos = exchange.get_positions()[stock_id_ref]
    unhedged_delta = total_delta_inst(exchange,stock_id_ref) + ref_pos
    
    for inst_id in list(insts_info.keys()):
        best_ask = insts_info[inst_id]['best_ask']
        best_bid = insts_info[inst_id]['best_bid']
        ideal_price = insts_info[inst_id]['ideal_ask_inst'] if delta_side == 'positive' else insts_info[inst_id]['ideal_bid_inst']
        inst_side = 'bid' if delta_side == 'positive' else 'ask'

        if (inst_id,inst_side) in unchanged_list: #the list tells us what limit orders not to send.
            continue
        
        if inst_id in puts:
            #if its put, then i want to produce + delta, then i want to try to sell puts, hence opposite
            ideal_price = insts_info[inst_id]['ideal_bid_inst'] if delta_side == 'positive' else insts_info[inst_id]['ideal_ask_inst']
            inst_side = 'ask' if delta_side == 'positive' else 'bid'

        inst_pos = insts_info[inst_id]['position']
        stock_price_tuple = (get_price(exchange, stock_id_ref,need_type ='bid'),get_price(exchange, stock_id_ref,need_type ='ask'))
        
        if delta_side == 'positive':
            #how much my position allows me to arb this price opp. always >=0
            position_opp,inst_bound,hedging_bound = position_opps(exchange, inst_pos, inst_id, ref_pos,stock_id_ref,stock_price_tuple,unhedged_delta, inst_side) 
            
            if inst_id not in puts:
                if  best_ask <= ideal_price:  # if my limit will be executed immediately, even i may have better price opp, i will still do it first.
                    arb_opp = ideal_price - best_ask  #price opportunity  (I expected to pay this, but can pay less!)
                    passive_list.append((1,arb_opp*position_opp,inst_id,best_ask,position_opp,inst_bound,hedging_bound,inst_side))
                else:
                    arb_opp = ideal_price - best_bid   #i join the queue. i prefer my limit bid to be much higher than market bid, since that means i am being most aggressive.
                    passive_list.append((0,arb_opp*position_opp,inst_id,ideal_price,position_opp,inst_bound,hedging_bound,inst_side))

            else: # if i am potentially inserting ask on puts to generate + delta.
                if  best_bid >= ideal_price:  # if i am putting on limit ask and i can sell higher than this:
                    arb_opp = best_bid - ideal_price   #price opportunity
                    passive_list.append((1,arb_opp*position_opp,inst_id,best_bid,position_opp,inst_bound,hedging_bound,inst_side))
                else:
#                    print(f'for {inst_id}, ideally we sell at {ideal_price}, current mkt bid is {best_bid}, mkt ask is {best_ask}')                    
                    arb_opp = best_ask - ideal_price   #i join the queue. i prefer my limit ask to be much lower than market ask, since that means i am being most aggressive.
                    passive_list.append((0,arb_opp*position_opp,inst_id,ideal_price,position_opp,inst_bound,hedging_bound,inst_side))
                                
        else:  #if im trying to generate negative delta on limit orders
            position_opp,inst_bound,hedging_bound = position_opps(exchange, inst_pos, inst_id, ref_pos,stock_id_ref,stock_price_tuple,unhedged_delta, inst_side) 
                        
            if inst_id not in puts:
                if  best_bid >= ideal_price:  # if i am putting on limit ask and i can sell higher than this:
                    arb_opp = best_bid - ideal_price   #price opportunity
                    passive_list.append((1,arb_opp*position_opp,inst_id,best_bid,position_opp,inst_bound,hedging_bound,inst_side))
                else:
                    arb_opp = best_ask - ideal_price   #i join the queue. i prefer my limit ask to be much lower than market ask, since that means i am being most aggressive.
                    passive_list.append((0,arb_opp*position_opp,inst_id,ideal_price,position_opp,inst_bound,hedging_bound,inst_side))
            else:#im potentially inserting bid on puts to generate -delta
                #the i ofc want to see if i can buy some puts
                if  best_ask <= ideal_price:  # if my limit will be executed immediately, even i may have better price opp, i will still do it first.
                    arb_opp = ideal_price - best_ask  #price opportunity 
                    passive_list.append((1,arb_opp*position_opp,inst_id,best_ask,position_opp,inst_bound,hedging_bound,inst_side))
                else:
                    arb_opp = ideal_price - best_bid   #i join the queue. i prefer my limit bid to be much higher than market bid, since that means i am being most aggressive.
                    passive_list.append((0,arb_opp*position_opp,inst_id,ideal_price,position_opp,inst_bound,hedging_bound,inst_side))                
    #sort based on size of arb opportunity                 
    #passive_list = sorted(passive_list,reverse=True)
    passive_list.sort(reverse=True)
    passive_list = [m[2:] for m in passive_list]

    return passive_list


def position_opps(exchange, inst_pos, inst_id, ref_pos, stock_id_ref,stock_price_tuple,unhedged_delta, inst_side):
    '''
    Calculator for calculating how much new limit order can I post this time. 
    Always >=0.
    Constrained by: 1. inst inventory  2. current inst order on other side  3. existing delta of derivs on same side
    unit_size also need to be given from overall 
    (maybe 40 limit orders to be divided among 10 instruments, but only top (5) will use it, then unit_size = 8)
    '''
    orders = exchange.get_outstanding_orders(inst_id)
    outstanding_orders_volume_bid = get_oustanding_orders_volume(orders,side = 'bid')
    outstanding_orders_volume_ask = get_oustanding_orders_volume(orders,side = 'ask')
    
    stock_value = stock_price_tuple[0] if inst_side =='bid' else stock_price_tuple[1]
    if inst_id in puts:
        #i try to bid puts, then i hedge by buying ref at ask. hence stock_value should be reversed for calculating the right delta
        stock_value = stock_price_tuple[1] if inst_side =='bid' else stock_price_tuple[0]
        
    #bound by inventory & current limit orders
    #if bid, i have 90, my opp is at most 10. if ask, i have -90, then my opp is at most 10.
    
    #bound by existing limit orders on same inst
    same_side_order = get_oustanding_orders_volume(orders,side = inst_side) 
    if inst_side == 'bid':
        inst_bound = 100 - inst_pos - same_side_order
    else:
        inst_bound = 100 + inst_pos - same_side_order
#    print(f'for {inst_id} {inst_side}, inst_pos = {inst_pos}, same_side_order = {same_side_order}')
    
    #also bound by other instruments & hedgeability
    if (inst_side =='bid' and inst_id in puts) or (inst_side =='ask' and inst_id not in puts):
        delta_side = 'negative' 
    else:
        delta_side = 'positive'
    
    #if all my outstanding orders that generates the same delta direction gets executed right now, how much delta does that produce
    worst_case_delta = total_delta_inst_outstanding(exchange,stock_id_ref,delta_side)
    #for positive worst case delta
    # i have outstanding delta of 20, i have ref_pos of -90. i can generate at most min(10,80)=10 extra (more)delta
    available_more_delta = min(100 - worst_case_delta, ref_pos + 100) - unhedged_delta
   
    #for negative worst case delta (i am trying to generate negative delta )
    # i have outstanding delta of 20; i have ref_pos of -90. i can generate at most max(-120,-190)=-120 extra (less) delta
    available_less_delta = max(-100 - worst_case_delta, ref_pos-100) - unhedged_delta
    units_per_delta = delta_to_units(exchange,inst_id, stock_value) #if negative
    if inst_side == 'bid':
        if units_per_delta >0:
            hedging_bound = math.floor(available_more_delta*units_per_delta)
        else: #for a negative delta instrument
            hedging_bound = math.ceil(available_less_delta*units_per_delta)
    else: 
        if units_per_delta >0: # for a positive delta inst, i can sell this much units
            hedging_bound = -math.ceil(available_less_delta*units_per_delta)
        else:#for a negative delta instrument, i can sell at most this much units
            hedging_bound = math.floor(available_more_delta*(-units_per_delta))

#    print(f'worst case outstnding delta is {round(worst_case_delta,1)}, where i try to generate {delta_side} delta')
    hedging_bound = max(hedging_bound,0) 
    
    overall_bound = min(inst_bound,hedging_bound)
    
    return overall_bound,inst_bound,hedging_bound

def send_limit_orders(exchange,time_set, passive_list,stock_price_tuple, delta_side,stock_id_ref,top_n = 5, new_delta_allocation = 20):
    '''
    passive_list is a list of lists of instructions to execute, max_unit always POSITIVE
    each sublist is [id,price,max_unit,'side']
    '''
    if delta_side == 'negative':
        new_delta_allocation = -new_delta_allocation
    top_n = min(top_n ,len(passive_list))
    per_unit_delta_allocation = new_delta_allocation/top_n
    used_delta = 0

    if delta_side == 'positive': #i'm trying to buy delta
        for each_inst in passive_list:
            inst_id,price,max_unit,inst_bound,hedging_bound,side = each_inst
            if max_unit<1:
                continue
            #check no self trade
            
            other_side = 'bid' if side == 'ask' else 'ask'
            orders = exchange.get_outstanding_orders(inst_id)
            other_price = get_oustanding_orders_price(orders,other_side)
            if other_price ==  price:
                continue
                
            #if i want to sell delta, then i hedge by buying ref at ask. hence use ask to calc deriv prices.   
            stock_value = stock_price_tuple[1]
            delta_per_unit = abs(calc_delta(exchange,inst_id, stock_value)) #even its put, i know im generating positive delta by selling it.
            
            #delta has been used by previous orders
            potential_delta_hedging_bound = delta_per_unit * hedging_bound - used_delta #this is a delta figure
            potential_delta_inst_bound = delta_per_unit * inst_bound 
            
            if potential_delta_hedging_bound < 1:
                break
            #volume is bound by 3 factors: delta i plan to use for this inst
            using_delta = min(per_unit_delta_allocation,potential_delta_hedging_bound,potential_delta_inst_bound)  
            using_units = math.floor(abs(using_delta / delta_per_unit))  #convert that to number of units
#            print(f'for {inst_id}, max_unit = {max_unit}, delta_inst_bound = {inst_bound}, delta_hedging_bound = {potential_delta_hedging_bound}, using_delta = {round(using_delta,1)}' )
                
            while True:
                cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                if cannot_trade:
                    continue #keep looping until i can trade
                break   
            exchange.insert_order(inst_id, price=price, volume=using_units, side=side, order_type='limit')
            _,time_set = dl.trade_would_breach_time_limit(time_set)
#            print(f'limit order {side} {using_units} units of {inst_id} at {price}')
            orders = exchange.get_outstanding_orders(inst_id)
            same_side_order = get_oustanding_orders_volume(orders,side = side) 
            cur_pos = exchange.get_positions()[inst_id]
#            print(f'after inserting, i have outstanding {side} of {same_side_order}, and my cur_pos is {cur_pos}')
            used_delta += using_delta
#            print(f'used_delta is {round(used_delta,1)}')
#            print(f'remaining delta is {round(new_delta_allocation - used_delta,1)}')
            if used_delta >= new_delta_allocation:
                #my new_delta_allocation hard limit doesn't allow me to trade more
                break
                
    else: #i'm trying to sell delta
        for each_inst in passive_list:
            inst_id,price,max_unit,inst_bound,hedging_bound,side = each_inst
            if max_unit<1:
                continue  
                
            other_side = 'bid' if side == 'ask' else 'ask'
            orders = exchange.get_outstanding_orders(inst_id)
            other_price = get_oustanding_orders_price(orders,other_side)
            if other_price ==  price:
                continue                
            
            stock_value = stock_price_tuple[0]            
            delta_per_unit = -abs(calc_delta(exchange,inst_id, stock_value))  #always negative delta if we are selling delta
            
            potential_delta_hedging_bound = delta_per_unit * hedging_bound - used_delta #negative delta figure
            potential_delta_inst_bound = delta_per_unit * inst_bound 
            
            if potential_delta_hedging_bound > -1:
                break
    
            #using _delta is always negeative here.
            using_delta = max(per_unit_delta_allocation,potential_delta_hedging_bound,potential_delta_inst_bound)  #delta i plan to use for this inst
            using_units = math.floor(using_delta / delta_per_unit)  #convert that to number of units
#            print(f'max_unit = {max_unit}, delta_inst_bound = {potential_delta_inst_bound}, delta_hedging_bound = {potential_delta_hedging_bound}, using_delta = {round(using_delta,1)}' )
           
        # insert order with time_set
            while True:
                cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                if cannot_trade:
                    continue #keep looping until i can trade
                break   
#            print(f'limit order {side} {using_units} units of {inst_id} at {price}')

            exchange.insert_order(inst_id, price=price, volume=using_units, side=side, order_type='limit')
            _,time_set = dl.trade_would_breach_time_limit(time_set)
            
            used_delta += using_delta
            if used_delta <= new_delta_allocation:
                break
    return time_set


def sort_passive(stock_id_ref, ref_pos):
    '''
    Determine if i want to increase/decrease delta first using limit orders 
    first check net delta, then check ref_pos. i want to balance delta first. 
    and then i prefer to have a small hedging inventory
    ''' 
    pos = ('positive','negative')
    neg = ('negative','positive')
    net_delta = total_delta_inst(exchange,stock_id_ref) + ref_pos
    
    if net_delta >=5:
        return neg
    elif net_delta <=-5:
        return pos
    elif ref_pos >0:
        return pos
    else:
        return neg
def cancel_prev_limits(exchange,time_set, insts_info):
    '''
    cancel prev orders on both sides if i have new price preferences
    '''
    unchanged_list = []
    
    for inst_id in list(insts_info.keys()):
        orders = exchange.get_outstanding_orders(inst_id)  
        for order in list(orders.values()):
            while True:
                cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                if cannot_trade:#keep looping until i can trade
                    continue
                break
            exchange.delete_order(instrument_id=inst_id,order_id = order.order_id)
            _,time_set = dl.trade_would_breach_time_limit(time_set)
    return [],time_set

## Active Strategy & Optimal execution

In [ ]:
def obtain_active_list(exchange,insts_info,stock_id_ref,delta_side = 'positive'):
    '''
    takes in the dictionary of dictionaries insts_info
    intended_side = buy or sell
    Returns a list of lists, where each sublist is [inst_id, best_available_price,inst_current_position,ioc_side]
    '''
    active_list = []
    inst_ids = list(insts_info.keys())
    
    ref_pos = exchange.get_positions()[stock_id_ref]    
    unhedged_delta = total_delta_inst(exchange,stock_id_ref) + ref_pos
    
    for inst_id in inst_ids:
        if (delta_side =='positive' and inst_id not in puts) or (delta_side =='negative' and inst_id in puts):
            inst_side = 'ask' #either i want +ve delta with normal, or negative delta by buying puts; then i want to buy stuff.
        else:
            inst_side = 'bid'  #otherwise, i want to sell it
            
        orders = exchange.get_outstanding_orders(inst_id)
        outstanding_volume = get_oustanding_orders_volume(orders,side = inst_side)
        if outstanding_volume == 0:#if i don't have outstanding bid when i want to CHECK bid side:
            #Then, i can check active trade possibility.
            best_price = insts_info[inst_id]['best_ask'] if inst_side == 'ask' else insts_info[inst_id]['best_bid']
            ideal_price = insts_info[inst_id]['ideal_ask_inst'] if inst_side == 'ask' else insts_info[inst_id]['ideal_bid_inst']
            if (best_price <= ideal_price) and (inst_side == 'ask'): #then i want to insert bid
                active_list.append((abs(ideal_price-best_price),inst_id,best_price,insts_info[inst_id]['position'],'bid'))
            elif (best_price >= ideal_price) and (inst_side == 'bid'):
                active_list.append((abs(ideal_price-best_price),inst_id,best_price,insts_info[inst_id]['position'],'ask'))
    
    #sort based on size of arb opportunity                 
    active_list = sorted(active_list,reverse=True)
    active_list = [[m[1],m[2],m[3],m[4]] for m in active_list]

    return active_list

def execute_trade_list(exchange, time_set,stock_id_ref, trade_list, delta_side ,max_total_delta =20):
    '''
    Takes in a list of trades to execute. [[id1,price,inst_pos,ioc_side],[...],...]
    Function is used in execute basket
    '''
    if delta_side == 'negative':
        max_total_delta = -max_total_delta

    each_inst_allocated_delta = round(max_total_delta/len(trade_list),0)
    list_copy= trade_list.copy()
    outstanding_insts_delta = total_delta_inst_outstanding(exchange,stock_id_ref,delta_side)

    for each_inst in trade_list:   
        #check invividual constraint
        inst_id,price,inst_pos,ioc_side = each_inst
        delta_per_unit = abs(calc_delta(exchange,inst_id))
        if delta_side == 'negative':
            delta_per_unit = -delta_per_unit
            
        inst_current_delta = delta_per_unit * inst_pos
        #def >=0
        each_inst_allocated_units = each_inst_allocated_delta / delta_per_unit 
        
        orders = exchange.get_outstanding_orders(inst_id)
        same_side_outstanding_volume = get_oustanding_orders_volume(orders,side = ioc_side)
        
        '''
        bound by unit limits
        '''
        #set extra_pos_inst always >=0 too.  
        if delta_side == 'positive':
            extra_pos_inst = min(each_inst_allocated_units, 100-inst_pos - same_side_outstanding_volume)
            if inst_id in puts:
                extra_pos_inst = min(each_inst_allocated_units, abs(-100 - inst_pos + same_side_outstanding_volume)) 
       
        else:  #i am selling stuff with +ve delta         we have inst_pos = -70, same side = 20, we can short at most -100+70+20=-10
            extra_pos_inst = abs(max(-each_inst_allocated_units, -100 - inst_pos + same_side_outstanding_volume))
            if inst_id in puts: #i am buying puts
                extra_pos_inst = min(abs(each_inst_allocated_units), 100-inst_pos - same_side_outstanding_volume)

        '''
        bound by delta limits
        '''
        #check overall book hedging constraint from other derivatives
        #if i have available buying/selling room left (i.e. number != 0 )
        if abs(extra_pos_inst)>=1: 
            current_insts_delta = total_delta_inst(exchange,stock_id_ref) #obtain current instruments delta
            inst_potential_delta = delta_per_unit * extra_pos_inst
            combined_delta = current_insts_delta+outstanding_insts_delta + inst_potential_delta
#            print(f'{(current_insts_delta,outstanding_insts_delta,inst_potential_delta)}')
            #check if new_combined_dela within is hedgeable:
            if (combined_delta <=100) and (combined_delta >= -100):
                #if hedgeable after trading, i shall trade, otherwise i trade partial
                if delta_side == 'positive':
                    #as long as everything togther is hedgeable, then this trade will be hedgeable.
                    hedging_bound_unit = abs((100-combined_delta)/delta_per_unit)  #100-combined_delta is def hedgeable for positive delta side. 
                else:
#                    print(f'combined delta is {combined_delta},delta/unit is {delta_per_unit}')
                    hedging_bound_unit = abs((100+combined_delta)/delta_per_unit)  #100-combined_delta is def hedgeable for positive delta side. 
                
                trading_pos = math.floor(min(extra_pos_inst,hedging_bound_unit))
                if trading_pos<1:
                    continue
                while True:
                    cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                    if cannot_trade:
                        continue #keep looping until i can trade
                    break
#                print(f'hedging bound unit is {hedging_bound_unit}, position_bound unit is {extra_pos_inst}')
                action = 'buy' if ioc_side=='bid' else 'sell'
#               print(f'Active {action}ing {inst_id} of {trading_pos} at price {price}')
                
                exchange.insert_order(
                    instrument_id=inst_id,
                    price=price,
                    volume=trading_pos,
                    side=ioc_side,
                    order_type='ioc')
                _,time_set = dl.trade_would_breach_time_limit(time_set)
                #i remove this from the original trade_list after execution
                list_copy.remove(each_inst)
    return list_copy,time_set

def execute_basket(exchange,time_set,insts_info, stock_id_ref, pos_list=[], neg_list=[],max_total_delta = 20): #execute as much as possible, but every execution has new delta <= 30
    '''
    1. each position bound by min(20, 100-own position)
    2. after execution, total new_delta <= hedgeable delta <= max_total
    buy/sell_list is [[id1,price,cur_pos],[id2,price,cur_pos]...]
    3. basket execution this way (3 times) allows spaces to be freeed up 
        for 2 sides and ensures most active trades gets done
    '''
    ref_pos = exchange.get_positions()[stock_id_ref]    
    if pos_list:
        pos_list, time_set = execute_trade_list(exchange, time_set,stock_id_ref, pos_list, delta_side='positive',max_total_delta = max_total_delta)
    if neg_list:
        neg_list, time_set = execute_trade_list(exchange, time_set, stock_id_ref,neg_list, delta_side='negative',max_total_delta = max_total_delta)
        if pos_list:
            pos_list, time_set = execute_trade_list(exchange, time_set, stock_id_ref,pos_list, delta_side='positive',max_total_delta = max_total_delta)

    return time_set
    

## Calc delta & prices

In [ ]:
def futures_fair_price(spot,FUTURE_ID,side = 'buy',int_rate=0.03):
    '''
    Calculates the fair price of a futures contract given spot. takes in a spot price and string FUTURE_ID
    if side= sell: i round up. it side = buy, i round down.
    '''
     # Hint: use the math.exp() function to implement the non-arbitrage relationship F = S * exp(r * tau)
    future_inst = exchange.get_instruments()[FUTURE_ID]
    expiry_date = future_inst.expiry
    time_to_expiry_yrs = libs.calculate_current_time_to_date(expiry_date)
    fair_price = spot * math.exp(int_rate * time_to_expiry_yrs)
    if side == 'sell':
        fair_price = math.ceil(fair_price*10)/10
    else:
        fair_price = math.floor(fair_price*10)/10
    return round(fair_price,1)

def fair_price(inst_id,price = 30,inst_type='f',side = 'buy'):
    '''
    if i am trying to buy, then I round down.
    
    '''
    if inst_id in futures:
        return futures_fair_price(price,FUTURE_ID = inst_id,side = side)
    elif inst_id in options:
        option = exchange.get_instruments()[inst_id]
        fair_price = oq.calculate_theoretical_option_value(expiry=option.expiry,
                                                           strike=option.strike,
                                                           option_kind=option.option_kind, 
                                                           stock_value=price,
                                                           interest_rate=0.03, 
                                                           volatility=3)
        if side == 'sell':
            fair_price = math.ceil(fair_price*10)/10
        else:
            fair_price = math.floor(fair_price*10)/10
        return fair_price
    else:
        return price
def get_price(exchange, inst_id,need_type = 'mid'):
    '''
    gets current price bid/ask/mid for any instrument
    '''
    while True:
        order_book_inst = exchange.get_last_price_book(inst_id)
        try:
            best_bid = order_book_inst.bids[0].price
            best_ask = order_book_inst.asks[0].price
        except:
            continue
        break
    if need_type == 'mid':
        price = round((best_bid + best_ask)/2,1)
    elif need_type == 'ask':
        price = round(best_ask,1)
    else:
        price = round(best_bid,1)
    return price

def get_derivs(stock_id_ref):
    '''
    returns all dual/future/options of a stock. as a list.
    '''
    derivs = []
    for inst in all_insts:
        if inst!= stock_id_ref and inst[:2] ==  stock_id_ref[:2]:
            derivs.append(inst)
            
    return derivs

def calc_delta(exchange, inst_id, stock_value = 30, interest_rate = 0.03, volatility=3):
    '''
    calculates unit delta of a deriv given stock price
    '''
    if inst_id in stocks:
        return 1
    else: 
        deriv_inst = exchange.get_instruments()[inst_id]
        expiry_date = deriv_inst.expiry
        time_to_expiry_yrs = libs.calculate_current_time_to_date(expiry_date)
        if inst_id in futures:
            delta = math.exp(interest_rate * time_to_expiry_yrs)
        else:
            strike = deriv_inst.strike
            option_kind = deriv_inst.option_kind
            delta = oq.calculate_option_delta(expiry_date, strike, option_kind, stock_value, interest_rate, volatility)
    return delta

def total_delta_inst(exchange,stock_id_ref):
    '''
    calculate total delta of derivativeson that stock
    '''
    derivs = get_derivs(stock_id_ref)
    total_delta = 0
    stock_value = get_price(exchange, stock_id_ref)
    
    for inst_id in derivs:
        delta = calc_delta(exchange,inst_id,stock_value = stock_value)
        if inst_id in futures:
            pos = exchange.get_positions()[inst_id]
            total_delta += delta * pos
        elif inst_id in options:
            pos = exchange.get_positions()[inst_id]
            total_delta += delta * pos
        else:
            pos = exchange.get_positions()[inst_id]
            total_delta += pos
    return total_delta

def total_delta_inst_outstanding(exchange,stock_id_ref,delta_side = 'positive'):
    '''
    calculate total oustanding delta of derivatives on that stock, given a delta_side
    i.e. we're trying to calculate worst case delta for a given delta shock direction
    '''
    derivs = get_derivs(stock_id_ref)
    total_delta = 0
    if delta_side == 'positive':
        for inst_id in derivs:
            orders = exchange.get_outstanding_orders(inst_id)
            outstanding_volume_bid = get_oustanding_orders_volume(orders,side = 'bid')
            outstanding_volume_ask = get_oustanding_orders_volume(orders,side = 'ask')    
            if inst_id in futures:
                delta = calc_delta(exchange,inst_id)
                total_delta += delta * outstanding_volume_bid
            elif inst_id in calls:
                stock_value_bid = get_price(exchange, stock_id_ref,need_type = 'bid')
                delta = calc_delta(exchange,inst_id,stock_value = stock_value_bid)
                total_delta += delta * outstanding_volume_bid
            elif inst_id in puts:
                stock_value_ask = get_price(exchange, stock_id_ref,need_type = 'ask')
                delta = calc_delta(exchange,inst_id,stock_value = stock_value_ask)
                total_delta += -delta * outstanding_volume_ask  #puts are negative delta
            else:
                total_delta += outstanding_volume_bid 
        return total_delta
    else:
        for inst_id in derivs:
            orders = exchange.get_outstanding_orders(inst_id)
            outstanding_volume_bid = get_oustanding_orders_volume(orders,side = 'bid')
            outstanding_volume_ask = get_oustanding_orders_volume(orders,side = 'ask')    
            if inst_id in futures:
                delta = calc_delta(exchange,inst_id)
                total_delta += delta * outstanding_volume_ask   #positive
            elif inst_id in calls:
                stock_value_ask = get_price(exchange, stock_id_ref,need_type = 'ask')
                delta = calc_delta(exchange,inst_id,stock_value = stock_value_ask)
                total_delta += delta * outstanding_volume_ask   #positive
            elif inst_id in puts:
                stock_value_bid = get_price(exchange, stock_id_ref,need_type = 'bid')
                delta = calc_delta(exchange,inst_id,stock_value = stock_value_bid)
                total_delta += -delta * outstanding_volume_bid  #puts are negative delta, hence make it positive too
            else:
                total_delta += outstanding_volume_bid 
        #this represents i will have negative delta if my asks with positive delta asks gets lifted and negative delta bids get hit
        return -total_delta
    
    

def delta_to_units(exchange, inst_id,stock_value = 34):
    '''
    1 delta corresponds to this many unit of derivatives
    '''
    delta = calc_delta(exchange = exchange,stock_value = stock_value, inst_id = inst_id)
    return round(1/delta,1)


## Helper functions

In [4]:
def get_oustanding_orders_volume(orders,side = 'both'):
    '''
    Calculates total orders I have ouststanding on the exchange. 
    is side = both, then i want both bid and ask.
    '''
    
    volume = 0 
    if side != 'both':
        for order in list(orders.values()):
            if order.side == side:
                volume += order.volume
        return volume
    else:
        for order in list(orders.values()):
            volume += order.volume
        return volume

def get_oustanding_orders_price(orders,side):
    '''
    Shows my current best bid/best ask on limit book.
    '''
    price = 99999 if side == 'ask' else -99999
    for order in list(orders.values()):
        if order.side == side:
            if side == 'ask':
                price = min(price,round(order.price,2))
            else:
                price = max(price,round(order.price,2))
    if abs(price)>9999:
        return 0.1
    return price


def cancel_one_side(exchange,time_set,inst_id, side):
    '''
    Cancels all orders on one side
    '''
    orders = exchange.get_outstanding_orders(inst_id)
    for order in list(orders.values()):
        if order.side == side:
            while True:
                cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                if cannot_trade:
                    continue #keep looping until i can trade
                break
            exchange.delete_order(instrument_id=inst_id,order_id = order.order_id)
            _,time_set = dl.trade_would_breach_time_limit(time_set)
            print(f'{inst_id} {side} order at {round(order.price,2)} deleted to allow for active trade')    
    
    return time_set

def show_oustanding_prices_volume_side(orders):
    '''
    returns a list of tuples for each order showing [(side,price,volume)]
    '''
    pv = []
    for order in list(orders.values()):
        pv.append((str(order.side),round(order.price,2),order.volume))
    return pv

def show_orders(exchange,inst_id):
    '''
    Shows current orders i have outstanding on the exchange. list of tuples.
    '''
    orderss = exchange.get_outstanding_orders(inst_id)
    side_price_volume = show_oustanding_prices_volume_side(orderss)
    print(f'current orders are: {side_price_volume}')
    
def record_position(older_pos,inst_id,ref_id):
    new_pos = {}
    pnl = exchange.get_pnl()
    timestamp = dt.datetime.now()
    
    positions = exchange.get_positions()
    current_position = positions[inst_id]
    ref_pos = positions[ref_id]
    
    net_delta = total_delta_inst(exchange,ref_id) + ref_pos

    new_pos['timestamp'] = timestamp     
    new_pos['pnl'] = pnl
    new_pos['net_delta'] = net_delta
    new_pos[f'{inst_id}_position'] = current_position
    new_pos[f'{ref_id}_position'] = ref_pos

    older_pos.append(new_pos)
    return older_pos


def record_df(older_pos):
    df= pd.DataFrame(older_pos).set_index('timestamp')
    return df

def show_analytics(df):
    time_dif = round((df.index[-1]- df.index[0]).total_seconds(),1)
    final_avg_pnl = df.pnl[-5:-1].mean()
    pnl_df = round(final_avg_pnl-df.pnl[0],1)
    avg_hr = round((pnl_df/time_dif) * 3600,1)
    print(f'hourly pnl is {avg_hr}USD; made {pnl_df}USD over {time_dif} seconds  ')
    
def clear_positions(exchange):
    for inst_id in all_insts:
        cur_pos = exchange.get_positions()[inst_id]
        if cur_pos != 0:
            dl.quick_order(exchange, instrument_id = inst_id,volume = -cur_pos)
    dl.print_positions_and_pnl(exchange = exchange, always_display=[stock_id_dual, stock_id_ref])
    
    
def delete_orders(exchange):
    for inst_id in all_insts:
        exchange.delete_orders(inst_id)
    print('all outstanding orders deleted')    

## Instrument Info

In [ ]:
%%time
def summarize_instruments(exchange,stock_id_ref,need_type='all'):
    '''
    Takes in a stock_id_ref and need_type, either 's','f' or 'o' for dif instruments
    Produces a dictionary of dictionaries: {inst_id_1: {'best_bid': x, 'best_ask': y,'position':P}, inst_id_2:...}
    We make sure we have least 1 ref + 1 derivative instrument
    '''
    while True:
        books = {}
        positions = exchange.get_positions()
        if need_type == 'all':
            if stock_id_ref == 'NVDA':
                inst_ids = NVDA_derivs
            else:
                inst_ids = SAN_derivs
        
        elif need_type == 's':
            if stock_id_ref == 'NVDA':
                inst_ids = ['NVDA_DUAL']
            else:
                inst_ids = ['SAN_DUAL']
        elif need_type == 'f':
            inst_ids = futures  
        elif need_type == 'o':
            inst_ids = options
        
        for each_id in inst_ids:
            '''
            Market orderbook info
            '''
            order_book_inst = exchange.get_last_price_book(each_id)
            try:
                best_bid_inst_price = round(order_book_inst.bids[0].price,2)
                best_ask_inst_price = round(order_book_inst.asks[0].price,2)
            except:
                best_bid_inst_price = 0.1
                best_ask_inst_price = 999
            current_pos_inst = positions[each_id]
            
            '''
            My outstanding order info
            '''
            outstanding_ask_price = None
            outstanding_bid_price = None

            orders = exchange.get_outstanding_orders(each_id)
            outstanding_orders_volume_ask = get_oustanding_orders_volume(orders,side='ask')
            if outstanding_orders_volume_ask!=0:
                outstanding_ask_price = get_oustanding_orders_price(orders,side = 'ask')
            outstanding_orders_volume_bid = get_oustanding_orders_volume(orders,side='bid')
            if outstanding_orders_volume_bid!=0:
                outstanding_bid_price = get_oustanding_orders_price(orders,side = 'bid')

            id_info = {'best_bid':best_bid_inst_price, 
                       'best_ask':best_ask_inst_price,
                       'position':current_pos_inst,
                       'outstanding_bid_volume':outstanding_orders_volume_bid,
                       'outstanding_ask_volume':outstanding_orders_volume_ask,
                       'outstanding_ask_price':outstanding_ask_price,
                       'outstanding_bid_price': outstanding_bid_price,
                      }
            books[each_id] = id_info
        
        if len(books)<1:
            continue
        break
    
    return books

def retreat_spread(current_inventory, hedging_inventory=0, min_spread = 0.1,is_put=False,is_SAN=False):  
    '''
    calculates my ideal spread based on my current inventory of the instrument & inventory of hedging stock
    if high inventory, i my spread is negative (lower price is easier to sell, harder to buy)
    however, if high hedging inventory, then i want to buy, so that i can sell my hedges. my spread will be positive. 
    Hence we have a conflict. 
    Decision: effect of inventory should be more important since it affects other instruments too.
    '''
    inst_inventory_effect = - min_spread * round(current_inventory/50,2)
    hedge_inventory_effect = min_spread * round(hedging_inventory/30,2)
    if is_put: #put has opposite hedge inventory effect
        hedge_inventory_effect = - hedge_inventory_effect
    if is_SAN:
        hedge_inventory_effect = hedge_inventory_effect + 0.2  #i observe i accumulate lots of SAN_DUAL
    my_spread =  inst_inventory_effect + hedge_inventory_effect
    return round(my_spread,1)


def add_ideal_prices(insts_info,stock_id_ref,ref_pos,default_spread = 0.2,strategy = 'active'):

    best_ask_ref_price = get_price(exchange, stock_id_ref,need_type ='ask')
    best_bid_ref_price = get_price(exchange, stock_id_ref,need_type ='bid')
    
    for inst_id,inst_info in insts_info.items():
        is_put = False
        is_SAN = False
        vol_skew = 0
        if inst_id in options:
            #i use vol_skew otherwise i find that i systematically short vol a lot
            vol_skew = 0.2
            if inst_id in puts:
                vol_skew +=0.1
                is_put = True
        if inst_id == 'SAN_DUAL':
            is_SAN = True
        ideal_ask_inst_price = round(fair_price(inst_id,best_bid_ref_price,side = 'buy') - default_spread + retreat_spread(inst_info['position'],ref_pos,is_put=is_put,is_SAN = is_SAN),1) + vol_skew
        ideal_bid_inst_price = round(fair_price(inst_id,best_ask_ref_price,side = 'sell') + default_spread + retreat_spread(inst_info['position'],ref_pos,is_put=is_put,is_SAN=is_SAN),1) + vol_skew
        
        if is_put:  
            ideal_ask_inst_price = round(fair_price(inst_id,best_ask_ref_price,side = 'buy') - default_spread + retreat_spread(inst_info['position'],ref_pos,is_put=is_put,is_SAN = is_SAN),1) + vol_skew
            ideal_bid_inst_price = round(fair_price(inst_id,best_bid_ref_price,side = 'sell') + default_spread + retreat_spread(inst_info['position'],ref_pos,is_put=is_put,is_SAN=is_SAN),1) + vol_skew
        
        if is_SAN:
            ideal_ask_inst_price -= 0.1
            ideal_bid_inst_price += 0.2
            
        inst_info['ideal_ask_inst'] = ideal_ask_inst_price if strategy == 'active' else round(ideal_ask_inst_price - 0.2,1)  #extra credit since limit order is riskier (me trying to buy)
        inst_info['ideal_bid_inst'] = ideal_bid_inst_price if strategy == 'active' else round(ideal_ask_inst_price + 0.2,1)  #extra credit since limit order is riskier (me trying to sell)
        
    return insts_info

# a = summarize_instruments(exchange,'NVDA')
# a = add_ideal_prices(a,'NVDA',0)
# a

## Hedging

In [ ]:
def hedge(exchange, stock_id_ref,ref_delta, time_set,target_delta = 0.0):
    
    current_insts_delta = total_delta_inst(exchange,stock_id_ref) #obtain current instruments delta
    delta = round(current_insts_delta + ref_delta,0)
    print(f'net delta before hedge is {delta}')
    if delta != target_delta:
        while True:
            while True:
                cannot_trade, time_set = dl.trade_would_breach_time_limit(time_set)
                if cannot_trade:
                    continue #keep looping until i can trade
                break
            order_book_ref = exchange.get_last_price_book(stock_id_ref)
            try: #somtimes there's no bid/ask on ref
                best_bid_ref_price = round(order_book_ref.bids[0].price,2)
                best_ask_ref_price = round(order_book_ref.asks[0].price,2)
            except:
                continue
            volume,side,price = (abs(delta),'bid',best_ask_ref_price) if delta<0 else (abs(delta),'ask',best_bid_ref_price)
            volume = round(volume,0)
            if delta<0:
                volume = min(volume,100-ref_delta)
            else: #i want to sell ref
                volume = min(volume,math.floor(abs(ref_delta+100)))
            if volume>0:
                action = 'buying' if side =='bid' else 'selling'
                print(f'Hedge by {action} {stock_id_ref}: {volume} lot(s) at price {price}.')
                exchange.insert_order(
                    instrument_id=stock_id_ref,
                    price=price,
                    volume=int(volume),
                    side=side,
                    order_type='ioc')
                _,time_set = dl.trade_would_breach_time_limit(time_set)
                break
            break

## Running Strategy

In [ ]:
stock_ids = ['NVDA','SAN']
older_pos = record_position([],'SAN','NVDA')
time_set = {}
while True:
    for stock_id_ref in stock_ids:
        order_book_ref = exchange.get_last_price_book(stock_id_ref)
        try:
            best_bid_ref_price = round(order_book_ref.bids[0].price,2)
            best_ask_ref_price = round(order_book_ref.asks[0].price,2)
        except:
            continue

        insts_info = summarize_instruments(exchange,stock_id_ref)  #includes current ask,bid,position for each id
        ref_pos = exchange.get_positions()[stock_id_ref]
        insts_info = add_ideal_prices(insts_info,stock_id_ref, ref_pos,strategy = 'active')

        _,time_set = cancel_prev_limits(exchange,time_set, insts_info)

        '''
        Sorting for Active Buy & Sells, and Actively trade the whole basket
        '''
        #obtain list of potential active trading candidates
        active_list_pos = obtain_active_list(exchange,insts_info,stock_id_ref,delta_side = 'positive') 
        active_list_neg = obtain_active_list(exchange,insts_info,stock_id_ref,delta_side = 'negative') 

        time_set = execute_basket(exchange,time_set,insts_info, stock_id_ref, active_list_pos, active_list_neg,max_total_delta = 20)


        '''
        Passive Putting up Orders
        '''

        insts_info = summarize_instruments(exchange,stock_id_ref)
        insts_info = add_ideal_prices(insts_info,stock_id_ref,ref_pos, strategy = 'passive')

        #don't send any limit orders contained in this list
        unchanged_list, time_set = cancel_prev_limits(exchange,time_set, insts_info)

        first_to_send,second_to_send = sort_passive(stock_id_ref, ref_pos)

        first_list = obtain_passive_list(exchange,unchanged_list,insts_info,stock_id_ref,first_to_send)      
        stock_price_tuple = (get_price(exchange, stock_id_ref,need_type ='bid'),get_price(exchange, stock_id_ref,need_type ='ask'))
        time_set = send_limit_orders(exchange,time_set, first_list,stock_price_tuple, first_to_send,stock_id_ref,top_n = 3, new_delta_allocation = 20)

        insts_info = summarize_instruments(exchange,stock_id_ref)
        insts_info = add_ideal_prices(insts_info,stock_id_ref,ref_pos, strategy = 'passive')    
        second_list = obtain_passive_list(exchange,unchanged_list,insts_info,stock_id_ref,second_to_send)
        time_set = send_limit_orders(exchange,time_set, second_list,stock_price_tuple, second_to_send,stock_id_ref,top_n = 3, new_delta_allocation = 20)

        '''
        Hedge
        '''
        target_delta = 0.0 
        hedge(exchange, stock_id_ref, ref_pos,time_set,target_delta)
    older_pos = record_position(older_pos,'SAN','NVDA')



In [ ]:
df = record_df(older_pos)
show_analytics(df)
df.pnl.plot()

In [ ]:
df.NVDA_position.plot()

In [ ]:
dl.print_positions_and_pnl(exchange = exchange, always_display=[stock_id_dual, stock_id_ref])